In [34]:
import pandas as pd
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

import warnings

warnings.filterwarnings('ignore')

In [35]:
df = pd.read_csv("income.csv",index_col=None)

In [36]:
df['target'].value_counts()

 <=50K     24720
 <=50K.    12435
 >50K       7841
 >50K.      3846
Name: target, dtype: int64

In [37]:
df['target'] = df['target'].str.replace('.','')

In [38]:
header = df.columns

In [39]:
le = preprocessing.LabelEncoder()

In [40]:
df['target']=le.fit_transform(df['target'])

In [47]:
num_cols = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']
ohe_cols = ['workclass','marital-status','occupation','relationship','race','sex','native-country']
target_transform_cols = ['education','occupation','native-country']